In [1]:
def show(title,image):
#     cv2.imshow(title,image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return

In [2]:
def one_check(x,y,w,h):
    wh_ratio = h / w

    if wh_ratio > 3:
        return True

    else:
        return False

In [3]:
def decimal_check(x,y,w,h):

    if w*h <= 150:
        return True
    else:
        return False

In [4]:
def pre_processing(image):
    image = imutils.resize(image,width=100)
    #image = cv2.medianBlur(image,3)
    image = cv2.GaussianBlur(image,(9,9),0)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    # opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(6,6))
    blackhat = cv2.morphologyEx(gray,cv2.MORPH_BLACKHAT,kernel)
    
#     show('image1',blackhat)
    
    _,thresh = cv2.threshold(blackhat,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
    thresh = cv2.dilate(thresh,None)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
    thresh = cv2.erode(thresh,kernel,iterations = 1)
    
    thresh = cv2.dilate(thresh,None)
    
#     show('image1',thresh)
     
    
    

#     gray= erosion

#     show('image1',gray)
    
#     _,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)

#     kernel = np.ones((2, 2), np.uint8)

#     thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
#     thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

#     thresh = ~thresh
    
    show('image1',thresh)
    
    return gray, thresh

# Blackhat
# Threshold
# Dilate
# Erode
# Dilate
# Erode

In [5]:
def greater(a, b):
    momA = cv2.moments(a)        
    (xa,ya) = int(momA['m10']/momA['m00']), int(momA['m01']/momA['m00'])

    momB = cv2.moments(b)        
    (xb,yb) = int(momB['m10']/momB['m00']), int(momB['m01']/momB['m00'])
    if xa > xb:
        return 1

    if xa == xb:
        return 0
    else:
        return -1

In [6]:
def extract_num(gray, thresh):
    (_,cnts,_) = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#      cnts = sorted(cnts, key=lambda x: greater(x[0], x[1])) 
#     cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0] + cv2.boundingRect(x)[1] * gray.shape[1] )
    cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])
    # x + y * w

    avgCntArea = np.mean([cv2.contourArea(k) for k in cnts])
    numbers = []
    rects = []
    
    for (i,c) in enumerate(cnts):
        mask = np.zeros(gray.shape,dtype="uint8")
        (x,y,w,h) = cv2.boundingRect(c)
        hull = cv2.convexHull(c)
        cv2.drawContours(mask,[hull],-1,255,-1)
        mask = cv2.bitwise_and(thresh,thresh,mask=mask)
        digit = mask[y-8:y+h+8,x-8:x+w+8]
        
        if digit.shape[0]!=0 and digit.shape[1]!=0:
            show('digit',digit)
    #         kernel = np.ones((1, 1), np.uint8)
    #         thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
#             show('digit',digit)
            digit = cv2.resize(digit,(28,28))
#             _,digit = cv2.threshold(digit,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
            show('digit',digit)
    #         kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1,1))
    #         digit = cv2.erode(digit,kernel,iterations = 3)
            show('digit',digit)
            rects.append((x,y,w,h))
            numbers.append(digit)
            
    return rects, numbers

In [7]:
def predict(cell_name):
    
    path = "/home/akshay/SEM6/IVP/cell extraction only/cells/"
#     path = ''
    str_num = ''
    one = 1
    
    image = cv2.imread(path + cell_name)
    
    gray, thresh = pre_processing(image)
          
    rects, numbers = extract_num(gray, thresh)
    
    image = imutils.resize(image,width=100)
    
    
    for i in range(len(numbers)):
        x = rects[i][0]
        y = rects[i][1]
        w = rects[i][2]
        h = rects[i][3]
        d_flag = decimal_check(x,y,w,h)
        one_flag = one_check(x,y,w,h)
        if d_flag:
            continue
        elif one_flag:
            num = one
            str_num += str(one)
        else:
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
            digit = cv2.erode(numbers[i],kernel,iterations = 1)
            digit = digit.reshape(1,28,28,1)
            prediction = new_model.predict(digit)
            num = np.argmax(prediction)
            str_num += str(num)
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),1)
        cv2.putText(image,str(num),(x+2,y-5),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,255,0),2,cv2.LINE_AA)
    
    show('image',image)
    return str_num
    

In [8]:
import numpy as np
import cv2
import imutils
import tensorflow as T

# new_model=T.keras.models.load_model('mnist_model_new.h5')

new_model=T.keras.models.load_model('/home/akshay/SEM6/IVP/Project/new_model.model')

super_l = []
l = []
str_num = ''

start_cell = 0
end_cell = 5
 
for i in range(0, 60):
    cell_name = str(i) + ".jpg"
#     cell_name = 'gray1.jpg'
    str_num = predict(cell_name)
    l.append(str_num)
    str_num = ''
    if (i+1) % (end_cell - start_cell + 1) == 0:
        super_l.append(l)
        l=[]
        
print(super_l)
print(l)

[['7', '10', '10', '10', '10', '7'], ['8', '9', '10', '9', '10', '3'], ['10', '10', '10', '10', '10', '9'], ['10', '10', '10', '10', '10', '8'], ['7', '10', '10', '9', '10', '8'], ['7', '10', '10', '9', '10', '5'], ['7', '10', '10', '18', '10', '7'], ['8', '10', '10', '10', '10', '9'], ['7', '10', '10', '7', '10', '1'], ['9', '10', '10', '9', '10', '7']]
[]
